In [ ]:
!pip install google-cloud-aiplatform
!pip install sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.7 MB/s eta 0:00:00


In [ ]:
import json
import faiss
import numpy as np
from vertexai.generative_models import GenerativeModel
from sentence_transformers import SentenceTransformer
from google.cloud import aiplatform
import os
import vertexai
from vertexai.language_models import TextGenerationModel


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
PROJECT_ID = "stunby-bangkit"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Sel 1: Load dan persiapkan dataset
def load_data(json_str):
  data = json.loads(json_str)
  data_list = data['data']
  contexts = [item['context'] for item in data_list]
  questions = [item['question'] for item in data_list]
  answers = [item['answer'] for item in data_list]
  return contexts, questions, answers

# Masukkan dataset Anda di sini
dataset_json = '''
{
  "data": [
    {
      "question": "Apa itu stunting?",
      "context": "Stunting adalah kondisi gagal tumbuh pada anak balita akibat kekurangan gizi kronis terutama dalam 1.000 hari pertama kehidupan. Kondisi ini ditandai dengan tinggi badan anak yang lebih pendek dibanding anak seusianya.",
      "answer": "Stunting adalah masalah pertumbuhan dimana anak memiliki tinggi badan yang lebih pendek dari standar usianya, yang disebabkan oleh kekurangan gizi kronis terutama pada 1.000 hari pertama kehidupan."
    },
    {
      "question": "Apa penyebab utama stunting?",
      "context": "Penyebab stunting meliputi: asupan gizi buruk selama kehamilan, kurangnya asupan protein, vitamin dan mineral pada anak, pola asuh yang tidak tepat, sanitasi buruk, dan kemiskinan.",
      "answer": "Penyebab utama stunting adalah kombinasi dari asupan gizi yang buruk selama kehamilan dan masa anak-anak, terutama kekurangan protein, vitamin, dan mineral penting. Ini juga dipengaruhi oleh pola asuh dan kondisi sanitasi lingkungan."
    },
    {
      "question": "Bagaimana cara mencegah stunting?",
      "context": "Pencegahan stunting dapat dilakukan melalui pemberian ASI eksklusif, makanan bergizi seimbang, pemantauan pertumbuhan rutin, sanitasi yang baik, dan pemberian suplemen gizi pada ibu hamil.",
      "answer": "Stunting dapat dicegah dengan beberapa cara kunci: memberikan ASI eksklusif selama 6 bulan pertama, menyediakan makanan bergizi seimbang, melakukan pemantauan pertumbuhan secara rutin, menjaga sanitasi lingkungan, dan memastikan ibu hamil mendapat asupan gizi yang cukup."
    },
    {
      "question": "Kapan periode kritis stunting?",
      "context": "Periode kritis stunting adalah 1.000 hari pertama kehidupan, dimulai dari masa kehamilan hingga anak berusia 2 tahun. Ini adalah periode emas pertumbuhan dan perkembangan anak.",
      "answer": "Periode kritis stunting adalah 1.000 hari pertama kehidupan, yang mencakup 9 bulan masa kehamilan dan 2 tahun pertama kehidupan anak. Ini adalah masa yang sangat penting karena menentukan kualitas kehidupan anak di masa depan."
    },
    {
      "question": "Apa dampak jangka panjang stunting?",
      "context": "Dampak jangka panjang stunting meliputi penurunan kemampuan kognitif, risiko penyakit metabolik, produktivitas kerja rendah, dan risiko melahirkan bayi dengan berat badan rendah.",
      "answer": "Stunting memiliki dampak jangka panjang yang serius, termasuk menurunnya kemampuan kognitif dan prestasi belajar, meningkatnya risiko penyakit metabolik di masa dewasa, rendahnya produktivitas kerja, serta risiko melahirkan bayi dengan berat badan rendah pada wanita yang mengalami stunting."
    },
    {
      "question": "Bagaimana cara mengukur stunting?",
      "context": "Stunting diukur dengan membandingkan tinggi badan anak dengan standar WHO untuk usia dan jenis kelamin yang sama. Anak dikategorikan stunting jika tinggi badannya kurang dari -2 SD dari standar median WHO.",
      "answer": "Stunting diukur dengan membandingkan tinggi badan anak terhadap standar pertumbuhan WHO. Jika tinggi badan anak berada di bawah -2 standar deviasi dari median standar WHO untuk usia dan jenis kelamin yang sama, maka anak tersebut dikategorikan mengalami stunting."
    },
    {
      "question": "Apa peran pemerintah dalam menangani stunting?",
      "context": "Pemerintah Indonesia memiliki program penanggulangan stunting melalui intervensi gizi spesifik dan sensitif, termasuk pemberian makanan tambahan, edukasi gizi, dan perbaikan sanitasi.",
      "answer": "Pemerintah berperan penting dalam menangani stunting melalui berbagai program intervensi, baik gizi spesifik maupun sensitif. Program ini mencakup pemberian makanan tambahan untuk ibu hamil dan balita, edukasi tentang gizi dan pola asuh, serta perbaikan akses terhadap air bersih dan sanitasi."
    },
    {
      "question": "Makanan apa yang baik untuk mencegah stunting?",
      "context": "Makanan pencegah stunting harus kaya protein, zat besi, zinc, vitamin A, dan kalsium. Contohnya: telur, ikan, daging, sayuran hijau, kacang-kacangan, dan susu.",
      "answer": "Untuk mencegah stunting, anak perlu mengonsumsi makanan kaya gizi seperti: sumber protein (telur, ikan, daging), sayuran hijau untuk zat besi dan vitamin, kacang-kacangan untuk protein nabati, dan produk susu untuk kalsium. Variasi makanan ini penting untuk memenuhi kebutuhan gizi anak."
    },
    {
      "question": "Apa tanda-tanda anak mengalami stunting?",
      "context": "Tanda stunting meliputi tinggi badan tidak sesuai usia, pertumbuhan terhambat, perkembangan terlambat, sering sakit, dan kemampuan kognitif kurang optimal.",
      "answer": "Tanda-tanda anak mengalami stunting dapat dilihat dari tinggi badan yang tidak sesuai dengan usianya, pertumbuhan yang lebih lambat dibanding anak seusianya, perkembangan motorik yang terlambat, sering mengalami sakit, dan kemampuan belajar yang kurang optimal."
    },
    {
      "question": "Apakah stunting bisa dipulihkan?",
      "context": "Pemulihan stunting paling efektif dilakukan sebelum usia 2 tahun. Setelah periode ini, dampak stunting lebih sulit dipulihkan namun masih bisa diminimalkan dengan intervensi gizi dan kesehatan yang tepat.",
      "answer": "Stunting paling efektif ditangani sebelum anak berusia 2 tahun, yang merupakan periode emas pertumbuhan. Setelah usia tersebut, pemulihan total lebih sulit dicapai, tetapi dampaknya masih bisa diminimalkan melalui perbaikan gizi dan perawatan kesehatan yang optimal."
    },
    {
      "question": "Apa itu KMS (Kartu Menuju Sehat)?",
      "context": "KMS adalah alat untuk memantau pertumbuhan anak yang berisi grafik pertumbuhan, catatan imunisasi, dan perkembangan anak dari lahir hingga usia 5 tahun.",
      "answer": "KMS adalah kartu yang digunakan untuk mencatat dan memantau pertumbuhan anak, termasuk berat badan, tinggi badan, imunisasi, dan perkembangan anak dari lahir sampai usia 5 tahun. KMS membantu orang tua dan petugas kesehatan mendeteksi masalah pertumbuhan sejak dini."
    },
    {
      "question": "Berapa kali sebaiknya anak ditimbang dalam sebulan?",
      "context": "Pemantauan pertumbuhan balita dilakukan minimal 1 kali sebulan di Posyandu untuk mendeteksi gangguan pertumbuhan secara dini.",
      "answer": "Anak sebaiknya ditimbang minimal 1 kali sebulan di Posyandu, terutama untuk anak usia di bawah 5 tahun. Penimbangan rutin ini penting untuk memantau pertumbuhan dan mendeteksi masalah pertumbuhan secara dini."
    },
    {
      "question": "Apa saja nutrisi penting untuk pertumbuhan anak?",
      "context": "Nutrisi penting meliputi protein, karbohidrat, lemak, vitamin A, D, zat besi, zinc, kalsium, dan asam folat yang mendukung pertumbuhan optimal.",
      "answer": "Nutrisi penting untuk pertumbuhan anak mencakup: protein untuk pembentukan sel dan jaringan, karbohidrat untuk energi, lemak untuk perkembangan otak, vitamin A untuk kekebalan tubuh, vitamin D untuk pertumbuhan tulang, zat besi untuk darah, zinc untuk pertumbuhan, kalsium untuk tulang dan gigi, serta asam folat untuk pembentukan sel."
    },
    {
      "question": "Bagaimana cara mengetahui porsi makan yang tepat untuk anak?",
      "context": "Porsi makan anak disesuaikan dengan usia, menggunakan pedoman gizi seimbang dengan prinsip 'isi piringku' yang terdiri dari karbohidrat, protein, sayur, dan buah.",
      "answer": "Porsi makan anak harus disesuaikan dengan usia. Gunakan prinsip 'isi piringku': 1/2 piring berisi karbohidrat (nasi/pengganti), 1/4 piring protein (ikan/daging/telur), dan 1/4 piring sayuran. Tambahkan buah sebagai pelengkap. Berikan porsi secara bertahap sesuai usia anak."
    },
    {
      "question": "Apa tanda-tanda anak kekurangan gizi?",
      "context": "Tanda kekurangan gizi meliputi berat badan kurang, pertumbuhan terhambat, mudah sakit, kurang aktif, rambut kusam, dan kulit kering.",
      "answer": "Tanda-tanda anak kekurangan gizi termasuk: berat badan di bawah standar usia, pertumbuhan yang melambat atau terhenti, mudah terserang penyakit, kurang energi dan aktivitas, rambut kusam kemerahan, dan kulit kering bersisik. Jika muncul tanda-tanda ini, segera konsultasi ke petugas kesehatan."
    },
    {
      "question": "Kapan waktu yang tepat untuk memperkenalkan MPASI?",
      "context": "MPASI diperkenalkan saat bayi berusia 6 bulan, dimulai dengan makanan tekstur lembut dan ditingkatkan secara bertahap sesuai perkembangan.",
      "answer": "MPASI (Makanan Pendamping ASI) mulai diperkenalkan saat bayi berusia 6 bulan, setelah periode ASI eksklusif. Mulai dengan makanan bertekstur lembut seperti bubur halus, kemudian tingkatkan teksturnya secara bertahap sesuai kemampuan bayi mengunyah."
    },
    {
      "question": "Bagaimana pola makan yang baik untuk mencegah stunting?",
      "context": "Pola makan untuk mencegah stunting harus memenuhi prinsip gizi seimbang, beragam, bergizi, dan aman, dengan frekuensi makan yang teratur.",
      "answer": "Pola makan yang baik untuk mencegah stunting meliputi: makan 3 kali sehari dengan 2 kali snack, memastikan setiap makanan mengandung protein, memberikan sayur dan buah setiap hari, memenuhi kebutuhan cairan, dan menjaga kebersihan makanan. Variasi menu penting untuk memenuhi semua kebutuhan nutrisi."
    },
    {
      "question": "Apa peran Posyandu dalam pemantauan pertumbuhan anak?",
      "context": "Posyandu berperan dalam pemantauan pertumbuhan melalui penimbangan, pengukuran tinggi badan, imunisasi, dan edukasi gizi kepada orang tua.",
      "answer": "Posyandu memiliki peran penting dalam pemantauan pertumbuhan anak melalui kegiatan rutin seperti penimbangan berat badan, pengukuran tinggi badan, pemberian imunisasi, dan memberikan edukasi gizi kepada orang tua. Posyandu juga membantu mendeteksi masalah pertumbuhan sejak dini."
    },
    {
      "question": "Bagaimana cara mencegah kekurangan zat besi pada anak?",
      "context": "Kekurangan zat besi pada anak dapat dicegah dengan memberikan makanan kaya zat besi seperti daging merah, hati ayam, bayam, kacang-kacangan, dan makanan yang diperkaya zat besi. Vitamin C juga membantu penyerapan zat besi.",
      "answer": "Untuk mencegah kekurangan zat besi pada anak, berikan makanan seperti daging merah, hati ayam, bayam, kacang-kacangan, serta makanan yang diperkaya zat besi. Pastikan makanan ini dikombinasikan dengan sumber vitamin C untuk membantu penyerapan zat besi."
    },
    {
      "question": "Apa peran vitamin D dalam pertumbuhan anak?",
      "context": "Vitamin D penting untuk pertumbuhan anak karena membantu penyerapan kalsium dan fosfor yang diperlukan untuk kesehatan tulang dan gigi. Kekurangan vitamin D dapat menyebabkan masalah pertumbuhan dan kesehatan tulang.",
      "answer": "Vitamin D berperan penting dalam mendukung pertumbuhan tulang dan gigi anak dengan meningkatkan penyerapan kalsium dan fosfor. Kekurangan vitamin D bisa menyebabkan rakitis atau gangguan pada tulang."
    },
    {
      "question": "Apa yang harus dilakukan jika anak tidak mau makan?",
      "context": "Jika anak tidak mau makan, langkah awal adalah mencari tahu penyebabnya, seperti rasa makanan, sakit, atau pola makan yang tidak teratur. Variasi makanan, porsi kecil, dan makan bersama keluarga dapat membantu meningkatkan nafsu makan anak.",
      "answer": "Ketika anak tidak mau makan, coba variasikan jenis makanan, sajikan dalam porsi kecil tapi sering, dan ajak anak makan bersama keluarga untuk menciptakan suasana yang menyenangkan. Jika masalah berlanjut, konsultasikan dengan dokter."
    },
    {
      "question": "Bagaimana peran air bersih dalam mencegah stunting?",
      "context": "Akses air bersih mengurangi risiko diare dan infeksi yang dapat menghambat penyerapan nutrisi. Sanitasi yang baik juga mendukung kesehatan anak dan mencegah kekurangan gizi yang bisa menyebabkan stunting.",
      "answer": "Air bersih sangat penting untuk mencegah stunting karena dapat mencegah infeksi seperti diare yang mengganggu penyerapan nutrisi. Pastikan keluarga memiliki akses ke air bersih dan sanitasi yang baik."
    },
    {
      "question": "Apa dampak stunting pada perkembangan otak anak?",
      "context": "Stunting dapat memengaruhi perkembangan otak karena kurangnya asupan nutrisi penting seperti omega-3, zat besi, dan vitamin B yang dibutuhkan untuk fungsi kognitif optimal.",
      "answer": "Stunting berdampak pada perkembangan otak anak karena kekurangan nutrisi penting yang mendukung fungsi kognitif, seperti omega-3, zat besi, dan vitamin B. Hal ini dapat memengaruhi kemampuan belajar dan perkembangan intelektual anak."
    },
    {
      "question": "Apa yang dimaksud dengan '1000 Hari Pertama Kehidupan' (HPK)?",
      "context": "1000 Hari Pertama Kehidupan adalah periode kritis dari hari pertama kehamilan hingga anak berusia 2 tahun. Pada periode ini, pertumbuhan fisik dan perkembangan otak anak berlangsung sangat cepat, sehingga asupan gizi sangat penting.",
      "answer": "1000 Hari Pertama Kehidupan (HPK) adalah periode dari kehamilan hingga anak berusia 2 tahun, yang merupakan waktu kritis untuk pertumbuhan fisik dan perkembangan otak. Pada periode ini, asupan gizi dan pola asuh yang baik sangat penting untuk mencegah stunting."
    },
    {
      "question": "Bagaimana cara meningkatkan asupan protein pada anak?",
      "context": "Asupan protein anak dapat ditingkatkan dengan memberikan makanan seperti telur, ikan, ayam, daging sapi, tahu, tempe, dan produk susu. Variasikan sumber protein agar anak tidak bosan.",
      "answer": "Tingkatkan asupan protein anak dengan memberikan makanan seperti telur, ikan, daging, tahu, tempe, dan susu. Variasi makanan penting untuk menjaga minat anak terhadap makanan."
    },
    {
      "question": "Apa itu makanan fortifikasi?",
      "context": "Makanan fortifikasi adalah makanan yang telah diperkaya dengan nutrisi tambahan seperti vitamin dan mineral yang bermanfaat untuk mencegah kekurangan gizi pada anak.",
      "answer": "Makanan fortifikasi adalah makanan yang diperkaya dengan nutrisi tambahan seperti zat besi, vitamin A, dan kalsium untuk mendukung kebutuhan gizi anak, terutama dalam pencegahan stunting."
    },
    {
      "question": "Bagaimana cara memastikan bayi mendapatkan ASI yang cukup?",
      "context": "Pastikan bayi menyusu setiap 2-3 jam atau sesuai kebutuhan. Perhatikan tanda bayi kenyang seperti terlihat tenang setelah menyusu dan berat badan bertambah secara normal.",
      "answer": "Untuk memastikan bayi mendapatkan ASI yang cukup, susui bayi setiap 2-3 jam dan perhatikan tanda-tanda bayi kenyang seperti terlihat tenang setelah menyusu, tidur nyenyak, dan berat badan yang bertambah sesuai dengan usia."
    },
    {
      "question": "Berapa prevalensi stunting di Indonesia?",
      "context": "Berdasarkan data Kementerian Kesehatan, prevalensi stunting di Indonesia masih cukup tinggi. Program pemerintah bertujuan menurunkan angka stunting hingga di bawah 14% pada tahun 2024.",
      "answer": "Prevalensi stunting di Indonesia masih menjadi tantangan kesehatan masyarakat yang signifikan. Pemerintah memiliki target untuk menurunkan angka stunting menjadi di bawah 14% pada tahun 2024 melalui berbagai program intervensi."
    },
    {
      "question": "Apa hubungan ASI eksklusif dengan stunting?",
      "context": "ASI eksklusif mengandung semua nutrisi yang dibutuhkan bayi dalam 6 bulan pertama kehidupan. ASI juga mengandung antibodi yang melindungi dari infeksi, sehingga mengurangi risiko stunting.",
      "answer": "ASI eksklusif berperan penting dalam mencegah stunting karena mengandung nutrisi lengkap yang dibutuhkan bayi selama 6 bulan pertama kehidupan. Selain itu, kandungan antibodi dalam ASI membantu melindungi bayi dari infeksi yang bisa menghambat pertumbuhan."
    },
    {
      "question": "Apa manfaat pemantauan pertumbuhan rutin?",
      "context": "Pemantauan pertumbuhan rutin membantu mendeteksi masalah pertumbuhan sejak dini, memastikan pemberian gizi tepat, dan mencegah stunting.",
      "answer": "Pemantauan pertumbuhan rutin bermanfaat untuk mendeteksi masalah pertumbuhan sejak awal, memastikan anak mendapat gizi yang sesuai kebutuhan, dan mencegah terjadinya stunting. Ini juga membantu orang tua dan petugas kesehatan mengambil tindakan cepat jika ada masalah."
    },
    {
      "question": "Bagaimana cara menyusun menu seimbang untuk anak?",
      "context": "Menu seimbang untuk anak harus mencakup karbohidrat, protein, sayuran, buah, dan susu, dengan porsi dan tekstur sesuai usia.",
      "answer": "Menyusun menu seimbang untuk anak dilakukan dengan memastikan setiap makanan mengandung karbohidrat (nasi/pengganti), protein (ikan/daging/telur), sayuran, dan buah. Tambahkan susu sebagai sumber kalsium. Sesuaikan porsi dan tekstur dengan usia anak."
    },
    {
      "question": "Bagaimana cara memantau pertumbuhan anak?",
      "context": "Pemantauan pertumbuhan dilakukan melalui pengukuran rutin tinggi dan berat badan di Posyandu, pencatatan di KMS, dan evaluasi perkembangan sesuai usia.",
      "answer": "Pertumbuhan anak dapat dipantau melalui kunjungan rutin ke Posyandu untuk pengukuran tinggi dan berat badan, pencatatan dalam Kartu Menuju Sehat (KMS), dan evaluasi perkembangan sesuai tahapan usia anak."
    },
    {
      "question": "Apa yang harus dilakukan jika anak terdeteksi berisiko stunting?",
      "context": "Jika terdeteksi berisiko stunting, diperlukan intervensi segera meliputi perbaikan pola makan, suplementasi gizi, dan konsultasi rutin dengan tenaga kesehatan.",
      "answer": "Jika anak terdeteksi berisiko stunting, langkah yang harus dilakukan adalah: segera konsultasi dengan tenaga kesehatan, perbaiki pola makan dengan meningkatkan asupan protein dan nutrisi penting, berikan suplementasi gizi sesuai anjuran, dan lakukan pemantauan pertumbuhan lebih intensif."
    }
  ]
}

'''

contexts, questions, answers = load_data(dataset_json)


In [ ]:
# Sel 2: Inisialisasi encoder dan buat embeddings
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
encoder = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Buat embeddings untuk konteks
context_embeddings = encoder.encode(contexts, convert_to_tensor=True)

# Inisialisasi FAISS index
# Ambil dimensi dari embeddings
# dimension = context_embeddings.shape[1](citation_1)
dimension = context_embeddings.shape[1]

# Inisialisasi FAISS index
index = faiss.IndexFlatL2(dimension)
index.add(context_embeddings.cpu().numpy())

In [ ]:
# Sel 3: Fungsi untuk mencari konteks yang relevan
def get_relevant_context(query, k=3):
    # Encode query
    query_vector = encoder.encode([query], convert_to_tensor=True)

    # Cari konteks yang paling mirip
    distances, indices = index.search(query_vector.cpu().numpy(), k)

    relevant_contexts = [contexts[idx] for idx in indices[0]]
    relevant_answers = [answers[idx] for idx in indices[0]]

    return relevant_contexts, relevant_answers


In [ ]:
# Sel 4: Fungsi chatbot menggunakan Gemini
def stunby_chatbot(query):
    # 1. Retrieval
    relevant_contexts, relevant_answers = get_relevant_context(query)

    # 2. Augmentation - menggabungkan konteks dengan query
    combined_context = " ".join(relevant_contexts)

    # 3. Generation menggunakan Gemini Pro
    model = GenerativeModel("gemini-1.5-pro-002")

    prompt = f"""
    Berdasarkan konteks berikut:
    {combined_context}

    Jawablah pertanyaan ini:
    {query}

    Berikan jawaban yang ringkas dan relevan.
    """

    generation_config = {
        "temperature": 0.7,
        "max_output_tokens": 1024,
        "top_p": 0.9,
        "top_k": 40
    }

    response = model.generate_content(
        prompt,
        generation_config=generation_config
    )

    return response.text


In [ ]:
# Sel 5: Simpan model dan index
import pickle

MODEL_DIR = os.path.join("models", "generative")

os.makedirs(MODEL_DIR, exist_ok=True)

def save_model():
    # Path lengkap untuk file
    index_path = os.path.join(MODEL_DIR, "stunby_index.faiss")
    data_path = os.path.join(MODEL_DIR, "stunby_data.pkl")

    # Simpan FAISS index
    faiss.write_index(index, index_path)

    # Simpan data pendukung
    data_to_save = {
        'contexts': contexts,
        'answers': answers,
        'encoder_name': model_name
    }

    # Simpan pickle file
    with open(data_path, 'wb') as f:
        pickle.dump(data_to_save, f)



In [ ]:
# Sel 6: Fungsi untuk load model
def load_stunby_model():
    index_path = os.path.join(MODEL_DIR, "stunby_index.faiss")
    data_path = os.path.join(MODEL_DIR, "stunby_data.pkl")
    index = faiss.read_index(index_path)

    with open(data_path, 'rb') as f:
        data = pickle.load(f)

    encoder = SentenceTransformer(data['encoder_name'])

    return index, encoder, data


In [ ]:
# Sel 7: Fungsi test
def test_chatbot():
    query = "Apa itu stuning?"
    response = stunby_chatbot(query)
    print(f"Q: {query}")
    print(f"A: {response}")


In [ ]:
if __name__ == "__main__":
    test_chatbot()

Q: Apa itu stuning?
A: Stunting adalah kondisi gagal tumbuh pada anak balita akibat kekurangan gizi kronis, terutama dalam 1.000 hari pertama kehidupan, ditandai dengan tinggi badan yang lebih pendek dibanding anak seusianya.



In [ ]:
save_model()